# Venue 86 - Thermopro loggers

Venue 86 was able to deploy a bunch of Thermpro TP357s at once.  

The green stripes are when the venue was occupied. The black squares are their records of the heating timings - from memory, so they may be imperfect, but it's brilliant to have these.

You can get a line to disappear by clicking on it in the legend, sweep out part of the plot to zoom in, and so on.

In [ ]:
# Using plotly.express

venue_string = "venue-86" 
# had to correct the times - Thermopro default export appears to have exactly the same format for am as pm so can only get from context. Did that in Excel by copying the first
# timestamp to column 4 as it was morning start, and then adding 1/24/4 to it for 15 minute intervals, filling down. 
# :TODO: if this is a common problem, need to specify the start timestamp in a constant here and add programmatically - but with some kind of check Thermopro never "skipped".

# import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go   


#from IPython.display import display

# now the homebuilt monitor 

rooms = ["1","2","3", "4","5", "6", "7"]
room_names= ["1 - lady_chapel","2 - north_aisle_rear","3 - right_back_pew", "4 - nave_aisle_central","5 - nave_aisle_near_front", "6 - choir", "7 - organ_loft"]


dfUseDiary = pd.read_csv(venue_string + '/use_diary.csv')
dfUseDiary = dfUseDiary.fillna('')
dfUseDiary['annotation_position'] = dfUseDiary['annotation_position'].replace(to_replace="", value="top left")
dfUseDiary['start'] = pd.to_datetime(dfUseDiary['startUse'])
dfUseDiary['end'] = pd.to_datetime(dfUseDiary['endUse'])
#print(dfUseDiary)

dfHeatTimes = pd.read_csv(venue_string + '/heating_timings.csv')
dfHeatTimes = dfHeatTimes.fillna('')
dfHeatTimes['start'] = pd.to_datetime(dfHeatTimes['startHeat'])
dfHeatTimes['end'] = pd.to_datetime(dfHeatTimes['endHeat'])

list_of_temp_traces = []
list_of_rh_traces = []

for room,name in zip(rooms,room_names):
    df = pd.read_csv("./" + venue_string + "/ThermoProSensor_export_" + room + " Red_09122024.csv",encoding='latin-1',usecols=[0,1,2], header=1, names=[ 'time','temperature','rh'])
    df["timestamp"] = pd.to_datetime(df['time'], dayfirst=True)
    df = df.fillna('')
    
    #range_min = df['timestamp'].min()
    #range_max = df['timestamp'].max()
    #print(range_min, " to ", range_max) # this just uses the range of the last file, not ideal
    
    
    trace = go.Scatter(customdata=df, 
                        y=df['temperature'], 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name=name,
                        )
    list_of_temp_traces.append(trace)
    
    trace = go.Scatter(customdata=df, 
                        y=df['rh'], 
                        x = df['timestamp'], 
                        mode='lines', 
                        hoverinfo='all', 
                        name=name,
                        )
    list_of_rh_traces.append(trace)

temp_g = go.FigureWidget(data=list_of_temp_traces,
                    layout = go.Layout(
                        yaxis=dict(range=[9,25])
                    ))

rh_g = go.FigureWidget(data=list_of_rh_traces,
                    layout = go.Layout(
                        yaxis=dict(range=[0,100])
                    ))



# example syntax for two plots on same x-axis - I'd like to show the boiler temperature in
# parallel - but havne't had time to sort the syntax.
#fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

# for i, col in enumerate(cols, start=1):
#     fig.add_trace(go.Scatter(x=df[col].index, y=df[col].values), row=i, col=1)

temp_fig = go.Figure(temp_g)
temp_fig.update_layout(showlegend=True, 
              autosize = True, 
              title= "Temperature (deg C))",
              width=1000, 
              height=500,
)

rh_fig = go.Figure(rh_g)
rh_fig.update_layout(showlegend=True, 
              autosize = True, 
              title= "Relative Humidity (%RH)",
              width=1000, 
              height=500,
)


temp_fig.update_layout(
    hovermode='x unified',
   # range=[range_min, range_max],
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

#Add range slider
temp_fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)

for ind in dfUseDiary.index:
    #print(dfCollatedUseDiaries['start'][ind])
    #print(dfCollatedUseDiaries['end'][ind]

    temp_fig.add_vrect(x0=dfUseDiary['start'][ind], x1=dfUseDiary['end'][ind],
                annotation_text=dfUseDiary['label'][ind], 
                annotation_position=dfUseDiary['annotation_position'][ind],
                annotation=dict(font_size=14, 
                font_family="Times New Roman"),
                fillcolor="green", 
                opacity=0.25, 
                line_width=0)

for ind in dfHeatTimes.index:
    #print(dfCollatedUseDiaries['start'][ind])
    #print(dfCollatedUseDiaries['end'][ind]

    temp_fig.add_vrect(x0=dfHeatTimes['start'][ind], x1=dfHeatTimes['end'][ind])
 
#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
temp_fig.show()

rh_fig.update_layout(
    hovermode='x unified',
   # range=[range_min, range_max],
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

#Add range slider
rh_fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)


#fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
rh_fig.show()
